In [128]:
import pandas as pd
import tensorflow as tf
import numpy as np
import math
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, LabelEncoder
from keras import models
from keras import layers
from keras import regularizers
import matplotlib.pyplot as plt
from keras import optimizers
from numpy.random import seed
from numpy import array
from numpy import argmax
from keras import metrics
import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model
from tensorflow.keras import optimizers

In [129]:
seed(10)
# read dataset
csv_file = 'assignment_ds.csv'
df = pd.read_csv(csv_file, header=None)
df = df.iloc[1: , :]
df[0:10]

,0,1,2,3,4,5,6
1,3453,0.000621,0.0773,17.08,Red,M,0
2,12098,689,7.01,0.02,Blue-white,A,3
3,2731,0.000437,0.0856,18.09,Red,M,0
4,37800,202900,6.86,-4.56,Blue,O,3
5,8570,0.00081,0.0097,14.2,Blue white,A,2
6,3605,126000,1124,-10.81,Red,M,5
7,8052,8.7,1.8,2.42,Whitish,A,3
8,3574,200000,89,-5.24,Red,M,4
9,8829,537493,1423,-10.73,White,A,5
10,3625,74000,876,-10.25,Red,M,5


In [130]:
df.shape

(200, 7)

In [131]:
# One Hot Encoding
df.groupby(4).apply(list)

4
Blue                  [0, 1, 2, 3, 4, 5, 6]
Blue White            [0, 1, 2, 3, 4, 5, 6]
Blue white            [0, 1, 2, 3, 4, 5, 6]
Blue-White            [0, 1, 2, 3, 4, 5, 6]
Blue-white            [0, 1, 2, 3, 4, 5, 6]
Orange                [0, 1, 2, 3, 4, 5, 6]
Orange-Red            [0, 1, 2, 3, 4, 5, 6]
Pale yellow orange    [0, 1, 2, 3, 4, 5, 6]
Red                   [0, 1, 2, 3, 4, 5, 6]
White                 [0, 1, 2, 3, 4, 5, 6]
White-Yellow          [0, 1, 2, 3, 4, 5, 6]
Whitish               [0, 1, 2, 3, 4, 5, 6]
Yellowish             [0, 1, 2, 3, 4, 5, 6]
Yellowish White       [0, 1, 2, 3, 4, 5, 6]
white                 [0, 1, 2, 3, 4, 5, 6]
yellow-white          [0, 1, 2, 3, 4, 5, 6]
yellowish             [0, 1, 2, 3, 4, 5, 6]
dtype: object

In [132]:
# column to lower case
df[4] = df[4].map(lambda x: x if type(x)!=str else x.lower())
df

,0,1,2,3,4,5,6
1,3453,0.000621,0.0773,17.08,red,M,0
2,12098,689,7.01,0.02,blue-white,A,3
3,2731,0.000437,0.0856,18.09,red,M,0
4,37800,202900,6.86,-4.56,blue,O,3
5,8570,0.00081,0.0097,14.2,blue white,A,2
...,...,...,...,...,...,...,...
196,13420,0.00059,0.00981,13.67,blue white,B,2
197,19860,0.0011,0.0131,11.34,blue,B,2
198,13023,998,6.21,-1.38,blue-white,A,3
199,39000,204000,10.6,-4.7,blue,O,3


In [133]:
df.groupby(4).apply(list)

4
blue                  [0, 1, 2, 3, 4, 5, 6]
blue white            [0, 1, 2, 3, 4, 5, 6]
blue-white            [0, 1, 2, 3, 4, 5, 6]
orange                [0, 1, 2, 3, 4, 5, 6]
orange-red            [0, 1, 2, 3, 4, 5, 6]
pale yellow orange    [0, 1, 2, 3, 4, 5, 6]
red                   [0, 1, 2, 3, 4, 5, 6]
white                 [0, 1, 2, 3, 4, 5, 6]
white-yellow          [0, 1, 2, 3, 4, 5, 6]
whitish               [0, 1, 2, 3, 4, 5, 6]
yellow-white          [0, 1, 2, 3, 4, 5, 6]
yellowish             [0, 1, 2, 3, 4, 5, 6]
yellowish white       [0, 1, 2, 3, 4, 5, 6]
dtype: object

In [134]:
df[4] = df[4].replace(['blue-white'], 'blue white')
df[4] = df[4].replace(['orange-red'], 'orange red')
# df[4] = df[4].replace(['yellow-white', 'yellowish', 'yellowish white'], 'yellow')
df[4] = df[4].replace(['yellow-white'], 'yellow white')
# df[4] = df[4].replace(['white-yellow', 'whitish'], 'white')
df[4] = df[4].replace(['white-yellow'], 'white yellow')
df

,0,1,2,3,4,5,6
1,3453,0.000621,0.0773,17.08,red,M,0
2,12098,689,7.01,0.02,blue white,A,3
3,2731,0.000437,0.0856,18.09,red,M,0
4,37800,202900,6.86,-4.56,blue,O,3
5,8570,0.00081,0.0097,14.2,blue white,A,2
...,...,...,...,...,...,...,...
196,13420,0.00059,0.00981,13.67,blue white,B,2
197,19860,0.0011,0.0131,11.34,blue,B,2
198,13023,998,6.21,-1.38,blue white,A,3
199,39000,204000,10.6,-4.7,blue,O,3


In [135]:
df.groupby(4).apply(list)

4
blue                  [0, 1, 2, 3, 4, 5, 6]
blue white            [0, 1, 2, 3, 4, 5, 6]
orange                [0, 1, 2, 3, 4, 5, 6]
orange red            [0, 1, 2, 3, 4, 5, 6]
pale yellow orange    [0, 1, 2, 3, 4, 5, 6]
red                   [0, 1, 2, 3, 4, 5, 6]
white                 [0, 1, 2, 3, 4, 5, 6]
white yellow          [0, 1, 2, 3, 4, 5, 6]
whitish               [0, 1, 2, 3, 4, 5, 6]
yellow white          [0, 1, 2, 3, 4, 5, 6]
yellowish             [0, 1, 2, 3, 4, 5, 6]
yellowish white       [0, 1, 2, 3, 4, 5, 6]
dtype: object

In [136]:
df.groupby(5).apply(list)

5
A    [0, 1, 2, 3, 4, 5, 6]
B    [0, 1, 2, 3, 4, 5, 6]
F    [0, 1, 2, 3, 4, 5, 6]
G    [0, 1, 2, 3, 4, 5, 6]
K    [0, 1, 2, 3, 4, 5, 6]
M    [0, 1, 2, 3, 4, 5, 6]
O    [0, 1, 2, 3, 4, 5, 6]
dtype: object

In [137]:
df

,0,1,2,3,4,5,6
1,3453,0.000621,0.0773,17.08,red,M,0
2,12098,689,7.01,0.02,blue white,A,3
3,2731,0.000437,0.0856,18.09,red,M,0
4,37800,202900,6.86,-4.56,blue,O,3
5,8570,0.00081,0.0097,14.2,blue white,A,2
...,...,...,...,...,...,...,...
196,13420,0.00059,0.00981,13.67,blue white,B,2
197,19860,0.0011,0.0131,11.34,blue,B,2
198,13023,998,6.21,-1.38,blue white,A,3
199,39000,204000,10.6,-4.7,blue,O,3


In [138]:
df = pd.get_dummies(df,columns=[4])
df

,0,1,2,3,5,6,4_blue,4_blue white,4_orange,4_orange red,4_pale yellow orange,4_red,4_white,4_white yellow,4_whitish,4_yellow white,4_yellowish,4_yellowish white
1,3453,0.000621,0.0773,17.08,M,0,0,0,0,0,0,1,0,0,0,0,0,0
2,12098,689,7.01,0.02,A,3,0,1,0,0,0,0,0,0,0,0,0,0
3,2731,0.000437,0.0856,18.09,M,0,0,0,0,0,0,1,0,0,0,0,0,0
4,37800,202900,6.86,-4.56,O,3,1,0,0,0,0,0,0,0,0,0,0,0
5,8570,0.00081,0.0097,14.2,A,2,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,13420,0.00059,0.00981,13.67,B,2,0,1,0,0,0,0,0,0,0,0,0,0
197,19860,0.0011,0.0131,11.34,B,2,1,0,0,0,0,0,0,0,0,0,0,0
198,13023,998,6.21,-1.38,A,3,0,1,0,0,0,0,0,0,0,0,0,0
199,39000,204000,10.6,-4.7,O,3,1,0,0,0,0,0,0,0,0,0,0,0


In [139]:
df = pd.get_dummies(df,columns=[5])
df

,0,1,2,3,6,4_blue,4_blue white,4_orange,4_orange red,4_pale yellow orange,...,4_yellow white,4_yellowish,4_yellowish white,5_A,5_B,5_F,5_G,5_K,5_M,5_O
1,3453,0.000621,0.0773,17.08,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,12098,689,7.01,0.02,3,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,2731,0.000437,0.0856,18.09,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,37800,202900,6.86,-4.56,3,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,8570,0.00081,0.0097,14.2,2,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,13420,0.00059,0.00981,13.67,2,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
197,19860,0.0011,0.0131,11.34,2,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
198,13023,998,6.21,-1.38,3,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
199,39000,204000,10.6,-4.7,3,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [140]:
cols = list(df.columns.values) #Make a list of all of the columns in the df
cols.pop(cols.index(6)) #Remove b from list
#cols.pop(cols.index('x')) #Remove x from list
df = df[cols+[6]] #Create new dataframe with columns in the order you want
df

,0,1,2,3,4_blue,4_blue white,4_orange,4_orange red,4_pale yellow orange,4_red,...,4_yellowish,4_yellowish white,5_A,5_B,5_F,5_G,5_K,5_M,5_O,6
1,3453,0.000621,0.0773,17.08,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
2,12098,689,7.01,0.02,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,3
3,2731,0.000437,0.0856,18.09,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
4,37800,202900,6.86,-4.56,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,3
5,8570,0.00081,0.0097,14.2,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,13420,0.00059,0.00981,13.67,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,2
197,19860,0.0011,0.0131,11.34,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,2
198,13023,998,6.21,-1.38,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,3
199,39000,204000,10.6,-4.7,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,3


In [141]:
df = df.rename(columns = {
    '4_blue': 4,
    '4_blue white':5,
    '4_orange':6,
    '4_orange red': 7,
    '4_pale yellow orange': 8,
    '4_red': 9,
    '4_white': 10,
    '4_white yellow': 11,
    '4_whitish': 12,
    '4_yellow white': 13,
    '4_yellowish':14,
    '4_yellowish white': 15,
    '5_A': 16,
    '5_B': 17,
    '5_F': 18,
    '5_G': 19,
    '5_K': 20,
    '5_M': 21,
    '5_O': 22,
    6: 23
}, inplace = False)
df

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
1,3453,0.000621,0.0773,17.08,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
2,12098,689,7.01,0.02,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,3
3,2731,0.000437,0.0856,18.09,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
4,37800,202900,6.86,-4.56,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,3
5,8570,0.00081,0.0097,14.2,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,13420,0.00059,0.00981,13.67,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,2
197,19860,0.0011,0.0131,11.34,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,2
198,13023,998,6.21,-1.38,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,3
199,39000,204000,10.6,-4.7,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,3


In [142]:
# shuffle dataset
df = df.sample(frac=1)

# get inputs - outputs
inputs = df[[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22]]
target = df [[23]]
print(inputs)
print(target)

        0         1       2       3   4   5   6   7   8   9   ...  13  14  15  \
60    3575    123000      45   -6.78   0   0   0   0   0   1  ...   0   0   0   
6     3605    126000    1124  -10.81   0   0   0   0   0   1  ...   0   0   0   
21   28700     16790     6.4   -4.09   0   1   0   0   0   0  ...   0   0   0   
199  39000    204000    10.6    -4.7   1   0   0   0   0   0  ...   0   0   0   
53    3042    0.0005  0.1542    16.6   0   0   0   0   0   1  ...   0   0   0   
..     ...       ...     ...     ...  ..  ..  ..  ..  ..  ..  ...  ..  ..  ..   
114   2831  0.000231  0.0915   16.21   0   0   0   0   0   1  ...   0   0   0   
65    3834    272000    1183    -9.2   0   0   0   0   0   1  ...   0   0   0   
16   17200   0.00098   0.015   12.45   0   1   0   0   0   0  ...   0   0   0   
126   3095   0.00019   0.492   10.87   0   0   0   0   0   1  ...   0   0   0   
10    3625     74000     876  -10.25   0   0   0   0   0   1  ...   0   0   0   

     16  17  18  19  20  21

In [143]:
# scale input data
x = inputs.values #returns a numpy array
min_max_scaler = MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
inputs = pd.DataFrame(x_scaled)
inputs[0:10]

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,0.042984,1.448047e-01,0.023090,0.160725,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.043772,1.483365e-01,0.576852,0.034709,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.703108,1.976643e-02,0.003280,0.244841,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.973726,2.401639e-01,0.005436,0.225766,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.028980,4.944550e-10,0.000075,0.891807,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5,0.017919,6.121824e-10,0.000046,0.773296,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
6,0.040750,3.178640e-01,0.780086,0.078799,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7,0.039227,6.945916e-10,0.000093,0.902439,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
8,0.017367,3.767276e-10,0.000045,0.916823,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
9,0.020809,4.202868e-10,0.000040,0.938399,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [144]:
# define training dataset size
ds_size = inputs.shape[0]
train_size_percentage = .8
test_size_percentage = .2

training_size = int(ds_size * train_size_percentage)
test_size = int(ds_size * test_size_percentage)

# get training data
input_training = inputs[:training_size]
target_training = target[:training_size]

# get test data
input_test = inputs[training_size:(training_size + test_size)]
target_test = target[training_size:(training_size + test_size)]

# used during training
print('Training DS size:', input_training.shape[0])
print('Test DS size:', input_test.shape[0])

Training DS size: 160
Test DS size: 40


In [149]:
print('inputs:')
print(inputs[:10])
print('target')
target[:10]

inputs:
           0             1         2         3    4    5    6    7    8    9   \
0    0.042984  1.448047e-01  0.023090  0.160725  0.0  0.0  0.0  0.0  0.0  1.0   
1    0.043772  1.483365e-01  0.576852  0.034709  0.0  0.0  0.0  0.0  0.0  1.0   
2    0.703108  1.976643e-02  0.003280  0.244841  0.0  1.0  0.0  0.0  0.0  0.0   
3    0.973726  2.401639e-01  0.005436  0.225766  1.0  0.0  0.0  0.0  0.0  0.0   
4    0.028980  4.944550e-10  0.000075  0.891807  0.0  0.0  0.0  0.0  0.0  1.0   
..        ...           ...       ...       ...  ...  ...  ...  ...  ...  ...   
195  0.023436  1.777684e-10  0.000043  0.879612  0.0  0.0  0.0  0.0  0.0  1.0   
196  0.049788  3.202185e-01  0.607132  0.085053  0.0  0.0  0.0  0.0  0.0  1.0   
197  0.400962  1.059547e-09  0.000003  0.762039  0.0  1.0  0.0  0.0  0.0  0.0   
198  0.030372  1.295001e-10  0.000248  0.712633  0.0  0.0  0.0  0.0  0.0  1.0   
199  0.044297  8.711827e-02  0.449574  0.052220  0.0  0.0  0.0  0.0  0.0  1.0   

     ...   13   14 

,23
60,4
6,5
21,3
199,3
53,0
...,...
114,0
65,5
16,2
126,1


In [146]:
# neural network
model = models.Sequential()

# input layer
model.add(layers.Dense(32, activation='relu', input_shape=(13,)))

# hidden layers
model.add(layers.Dense(16, activation='relu'))

# output layers
model.add(layers.Dense(3))

# rmsprop = keras.optimizers.Adam(learning_rate=0.1)
rmsprop = optimizers.RMSprop(learning_rate=0.1)
model.compile(optimizer=rmsprop, loss='mse', metrics=[metrics.categorical_accuracy])

In [151]:
#df.shape[1]
# df.shape[0]
#df[23]
print(input_training)
print(input_test)

           0             1             2         3    4    5    6    7    8   \
0    0.042984  1.448047e-01  2.309048e-02  0.160725  0.0  0.0  0.0  0.0  0.0   
1    0.043772  1.483365e-01  5.768522e-01  0.034709  0.0  0.0  0.0  0.0  0.0   
2    0.703108  1.976643e-02  3.280281e-03  0.244841  0.0  1.0  0.0  0.0  0.0   
3    0.973726  2.401639e-01  5.435795e-03  0.225766  1.0  0.0  0.0  0.0  0.0   
4    0.028980  4.944550e-10  7.482711e-05  0.891807  0.0  0.0  0.0  0.0  0.0   
..        ...           ...           ...       ...  ...  ...  ...  ...  ...   
155  0.183521  2.354548e-10  2.001548e-07  0.837711  0.0  1.0  0.0  0.0  0.0   
156  0.032606  1.083092e-09  1.753151e-04  0.740463  0.0  0.0  0.0  0.0  0.0   
157  0.801897  2.046102e-01  3.196627e-03  0.236398  1.0  0.0  0.0  0.0  0.0   
158  0.429600  1.436274e-09  4.824245e-07  0.772358  1.0  0.0  0.0  0.0  0.0   
159  0.266966  1.412729e-01  3.638156e-01  0.127580  0.0  1.0  0.0  0.0  0.0   

      9   ...   13   14   15   16   17 

In [148]:
model.fit(inputs, target, epochs=1000)

Epoch 1/1000


ValueError: in user code:

    File "C:\Users\Adrian\anaconda3\lib\site-packages\keras\engine\training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\Adrian\anaconda3\lib\site-packages\keras\engine\training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Adrian\anaconda3\lib\site-packages\keras\engine\training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\Adrian\anaconda3\lib\site-packages\keras\engine\training.py", line 808, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\Adrian\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\Adrian\anaconda3\lib\site-packages\keras\engine\input_spec.py", line 263, in assert_input_compatibility
        raise ValueError(f'Input {input_index} of